In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.
┌ Info: Precompiling GraphPPL [b3f8163a-e979-4e85-b43e-1f63d8c8b42c]
└ @ Base loading.jl:1278
┌ Warning: Package GraphPPL does not have ReactiveMP in its dependencies:
│ - If you have GraphPPL checked out for development and have
│   added ReactiveMP as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with GraphPPL
└ Loading ReactiveMP into GraphPPL from project dependency, future 

In [2]:
@model [ outbound_message_portal = EmptyPortal() ] function smoothing(n, k, x0, P)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ (x_prev + 1.0) where { portal = rem(i, k) === 0 ? AsyncPortal() : EmptyPortal() }
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
P = 1.0

n = 10_000
k = 500
data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
function inference(; data, k, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, k, x0, P);

    buffer    = Vector{Marginal}(undef, n)
    marginals = collectLatest(getmarginals(x))
    
    subscription = subscribe!(marginals, (ms) -> copyto!(buffer, ms))
    
    yield()
    
    update!(y, data)
    
    for i in 1:(div(n, k) + 1)
        yield()
    end
    
    unsubscribe!(subscription)
    
    return buffer
end

inference (generic function with 1 method)

In [6]:
@time res = inference(
    data = data,
    k = k,
    x0 = NormalMeanVariance(0.0, 10000.0),
    P = P
)

  0.916670 seconds (7.08 M allocations: 441.416 MiB, 38.02% gc time)


10000-element Array{Marginal,1}:
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=0.9895455386852966, v=9.999999899999937e-5))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=1.9895455386852967, v=9.999999899999937e-5))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=2.989545538685296, v=9.999999899999936e-5))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=3.9895455386852965, v=9.999999899999936e-5))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=4.989545538685298, v=9.999999899999937e-5))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=5.989545538685298, v=9.999999899999937e-5))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=6.989545538685299, v=9.999999899999939e-5))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Float64}(μ=7.989545538685296, v=9.999999899999934e-5))
 Marginal{NormalMeanVariance{Float64}}(NormalMeanVariance{Fl